# Mine Safety

We're interested in [US mine safety](https://arlweb.msha.gov/drs/drshome.htm#MID), thank goodness we can search for these things.

## Preparation: Knowing your tags

These questions are the same for every data set, and might not work exactly for yours.

**Search for every operator with 'dirt' in their name, including abandoned mines.**

### What is the tag and class name for every row of data?

In [ ]:
#tr

### What is the tag and class name for every mine operator's name?

In [3]:
#td[2]

### What is the tag and class name for every mine's name?

In [ ]:
#td[3]

## Being lazy

If you only needed these results, what would you do instead of scraping them?

## Setup: Import what you'll need to scrape the page

Use `requests`, not `urllib`.

In [49]:
import requests
from bs4 import BeautifulSoup

## Try to scrape the page

To test if you requested the page correctly, save the BeautifulSoup document as `doc` and run the code `doc.find_all('tr')[-1].text` to get the text of the last `<tr>` element.

- If the result starts with **Total Number of Mines Found**, you were successful.

In [50]:
data = {
    'OperSearch':'dirt',
    'Abandoned':'No',
    'MineName':'',
    'StateSearch':'None',
    'CM':'All',
    'x':'36',
    'y':'4',
    'MC':'Opersearch',
}
response = requests.post("https://arlweb.msha.gov/drs/ASP/OprNameStatesearch.asp", data=data)
doc = BeautifulSoup(response.text, 'html.parser')

In [51]:
doc.find_all('tr')[-1]

<tr>
<td bgcolor="#000000" colspan="8"><font style="FONT-SIZE:.75em; color:#FFF"><b>Total Number of Mines Found:</b>  <b>19</b></font></td></tr>

## Actually scraping

### Hopefully you know that each `tr` is supposed to be your data. What is the index of the first row element that is actually a result?

`.text` will help you here.

In [52]:
doc.find_all('tr')[7]

<tr>
<td align="center">
<form action="/drs/ASP/BasicMineInfostatecounty.asp" method="post" name="search">
<input name="MineId" type="hidden" value="3503598"/><font style="FONT-SIZE:.75em;">3503598</font>
</form></td>
<td><font style="FONT-SIZE:.75em;"><!-- DNT --><b>OR</b><!-- /DNT --> </font></td>
<td><font style="FONT-SIZE:.75em;"><!-- DNT --> Newberg Rock &amp; Dirt<!-- /DNT -->  </font></td>
<td><font style="FONT-SIZE:.75em;"><!-- DNT -->Newberg Rock &amp; Dirt<!-- /DNT --></font></td>
<td align="center"><font style="FONT-SIZE:.75em;"><!-- DNT -->Surface             <!-- /DNT --></font></td>
<td align="center"><font style="FONT-SIZE:.75em;"><!-- DNT -->M<!-- /DNT --> </font></td>
<td><font style="FONT-SIZE:.75em;">Active  </font></td>
<td><font style="FONT-SIZE:.75em;">Crushed, Broken Stone NEC  </font></td>
<th bgcolor="#000000"><input alt="More Information" border="0" name="submit" src="/drs/images/moreinfo.jpg" type="image"/></th></tr>

### Loop through each operator result, printing its name

Use LIST SLICING to skip the non-data row(s).

In [57]:
for name in doc.find_all('tr')[7:-1]:
    print(name.find_all('td')[2].text.strip())

Newberg Rock & Dirt
AM Dirtworks & Aggregate Sales
Dirt Company
Dirt Con
Dirt Doctor Inc
Dirt Works
Holley Dirt Company, Inc
Krueger Brothers Gravel & Dirt
M R Dirt
M.R. Dirt Inc.
P B Dirt Movers, Inc
PB Dirt Movers
PB Dirt Movers, Inc
Prescott Dirt, LLC
R D Blankenship Dirt Work LLC
SIMPSON DIRTWORX LLC
SIMPSON DIRTWORX LLC
Spry's Dirt & Gravel, Inc.
Vogt Dirt Service


### Loop through each operator result, printing its ID

There should be ONE code per row, and NO empty rows between them.

In [58]:
for ID in doc.find_all('tr')[7:-1]:
    print(ID.find_all('td')[0].text.strip())

3503598
4801789
5001797
4608254
2103723
4104757
0801306
3901432
3609624
3609931
1519799
4407296
4407270
0203332
2901986
4300768
4300776
2302283
2103518


## Saving the results

### Loop through each `tr` to create a list of dictionaries

Each dictionary must contain

- Operator ID
- Operator name
- Mine name
- State
- Mine type
- Coal or metal
- Status
- Commodity

Create a new dictionary for each row.

In [74]:
dirt_mines = []
for mine in doc.find_all('tr')[7:-1]:
    dirt_mines.append({
        'ID': mine.find_all('td')[0].text.strip(),
        'State': mine.find_all('td')[1].text.strip(),
        'Operator': mine.find_all('td')[2].text.strip(),
        'Mine name': mine.find_all('td')[3].text.strip(),
        'type': mine.find_all('td')[4].text.strip(),
        'CM': mine.find_all('td')[5].text.strip(),
        'Status': mine.find_all('td')[6].text.strip(),
        'Commodity': mine.find_all('td')[7].text.strip(),
    })

### Save that to a CSV

In [75]:
import pandas as pd

In [76]:
df = pd.DataFrame(dirt_mines)
df.to_csv('dirt_mines.csv', index=False)

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.

In [77]:
pd.read_csv('dirt_mines.csv')

,CM,Commodity,ID,Mine name,Operator,State,Status,type
0,M,"Crushed, Broken Stone NEC",3503598,Newberg Rock & Dirt,Newberg Rock & Dirt,OR,Active,Surface
1,M,Construction Sand and Gravel,4801789,AM Dirtworks & Aggregate Sales,AM Dirtworks & Aggregate Sales,ND,Intermittent,Surface
2,M,Construction Sand and Gravel,5001797,Bush Pilot,Dirt Company,AK,Intermittent,Surface
3,M,"Crushed, Broken Limestone NEC",4608254,Hog Lick Quarry,Dirt Con,WV,Temporarily Idled,Surface
4,M,Construction Sand and Gravel,2103723,Rock Lake Plant,Dirt Doctor Inc,MN,Intermittent,Surface
5,M,Construction Sand and Gravel,4104757,Portable #1,Dirt Works,TX,Intermittent,Surface
6,M,"Sand, Common",801306,River Road Pit,"Holley Dirt Company, Inc",FL,Active,Surface
7,M,Construction Sand and Gravel,3901432,PORTABLE SCREENER,Krueger Brothers Gravel & Dirt,SD,Intermittent,Surface
8,M,Construction Sand and Gravel,3609624,Forbes Pit,M R Dirt,PA,Intermittent,Surface
9,M,Dimension Stone NEC,3609931,Camptown Quarry,M.R. Dirt Inc.,PA,Intermittent,Surface
